In [1]:



import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt

# Step 1: Load the dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv'
dataframe = pd.read_csv(url, usecols=[1], engine='python')
dataset = dataframe.values.astype('float32')

# Step 2: Prepare the data
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

# Split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]

# Convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# Reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

# Step 3: Build the LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Step 4: Train the model
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

# Step 5: Evaluate the model
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# Invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

# Calculate error scores
trainScore = mean_absolute_error(trainY[0], trainPredict[:,0])
print(f'Train Score: {trainScore:.2f} MAE')
testScore = mean_absolute_error(testY[0], testPredict[:,0])
print(f'Test Score: {testScore:.2f} MAE')

# RMSE can be calculated similarly using mean_squared_error with squared=False

# Plotting baseline and predictions
plt.figure(figsize=(10,6))
plt.plot(scaler.inverse_transform(dataset))
plt.plot(np.concatenate([np.empty((look_back+1))*np.nan, trainPredict[:,0]]))
plt.plot(np.concatenate([np.empty((len(train)+look_back+1))*np.nan, testPredict[:,0]]))
plt.show()

C:\Users\korem\AppData\Local\Temp\ipykernel_30456\1927374377.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
import numpy as np
from keras.datasets import mnist

# Load MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize pixel values to [0, 1]
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))  # 10 output classes (digits 0-9)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images[..., np.newaxis], train_labels, epochs=5, batch_size=64)
test_loss, test_acc = model.evaluate(test_images[..., np.newaxis], test_labels)
print(f"Test accuracy: {test_acc:.4f}")
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

def create_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

keras_model = KerasClassifier(build_fn=create_model, verbose=0)
param_grid = {
    'batch_size': [32, 64],
    'epochs': [5, 10],
    'optimizer': ['adam', 'rmsprop'],
    'learning_rate': [0.001, 0.01]
}

grid_search = GridSearchCV(estimator=keras_model, param_grid=param_grid, cv=3)
grid_result = grid_search.fit(train_images[..., np.newaxis], train_labels)

print(f"Best parameters: {grid_result.best_params_}")
print(f"Best accuracy: {grid_result.best_score_:.4f}")
from keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)
history = model.fit(train_images[..., np.newaxis], train_labels, epochs=10, batch_size=64, validation_split=0.2, callbacks=[reduce_lr])
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


Epoch 1/5
938/938 [==============================] - 30s 31ms/step - loss: 0.1773 - accuracy: 0.9491
Epoch 2/5
938/938 [==============================] - 29s 31ms/step - loss: 0.0573 - accuracy: 0.9830
Epoch 3/5
938/938 [==============================] - 30s 32ms/step - loss: 0.0364 - accuracy: 0.9893
Epoch 4/5
938/938 [==============================] - 29s 31ms/step - loss: 0.0267 - accuracy: 0.9917
Epoch 5/5
313/313 [==============================] - 2s 6ms/step - loss: 0.0495 - accuracy: 0.9845
Test accuracy: 0.9845


ModuleNotFoundError: No module named 'keras.wrappers'

In [ ]:
import numpy as np
from keras.datasets import mnist
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt

# Load MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize pixel values to [0, 1]
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))  # 10 output classes (digits 0-9)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images[..., np.newaxis], train_labels, epochs=5, batch_size=64)
test_loss, test_acc = model.evaluate(test_images[..., np.newaxis], test_labels)
print(f"Test accuracy: {test_acc:.4f}")

def create_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

keras_model = KerasClassifier(build_fn=create_model, verbose=0)
param_grid = {
    'batch_size': [32, 64],
    'epochs': [5, 10],
    'optimizer': ['adam', 'rmsprop'],
    'learning_rate': [0.001, 0.01]
}

grid_search = GridSearchCV(estimator=keras_model, param_grid=param_grid, cv=3)
grid_result = grid_search.fit(train_images[..., np.newaxis], train_labels)

print(f"Best parameters: {grid_result.best_params_}")
print(f"Best accuracy: {grid_result.best_score_:.4f}")

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)
history = model.fit(train_images[..., np.newaxis], train_labels, epochs=10, batch_size=64, validation_split=0.2, callbacks=[reduce_lr])

plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


ModuleNotFoundError: No module named 'tensorflow.keras.wrappers'